In [44]:
import random
import numpy as np

class NaiveSodukuGenerator:
	def __init__(self) -> None:
		self.pz = np.zeros((9,9), dtype=int)
		
	def PossibleValueAtPosition(self, row:int, col:int):
		r = row//3*3
		c = col//3*3
		return {1,2,3,4,5,6,7,8,9}.difference(set(self.pz[r:r+3,c:c+3].flat\
			)).difference(set(self.pz[row,:])).difference(set(self.pz[:,col]))

	def Solution_Count(self, pz, n:int, Nof_solution:int):
		if Nof_solution > 1:
			return Nof_solution
		if n>=81:
			Nof_solution+=1
			return Nof_solution
		(row,col) = divmod(n,9)
		if pz[row][col] > 0:
			Nof_solution = self.Solution_Count(pz, n+1, Nof_solution)
		else:
			l = self.PossibleValueAtPosition( row,col)
			for v in l:
				pz[row][col] = v
				Nof_solution = self.Solution_Count(pz, n+1, Nof_solution)
				pz[row][col] = 0
		
		return Nof_solution	

	def SudokuSolver(self, n:int):
		if n==81:
			return True
		(row,col) = divmod(n,9)
		if self.pz[row][col]>0:
			if self.SudokuSolver(self.pz, n+1):
				return True
		else:
			l = list(self.PossibleValueAtPosition( row,col))
			random.shuffle(l)
			for v in l:
				self.pz[row][col] = v
				if self.SudokuSolver(n+1):
					return True
				self.pz[row][col] = 0
		return False

	def DigHoles(self,  randomlist, n:int):
		if n >= 81:
			return
		(row,col) = divmod(randomlist[n],9)
		if self.pz[row][col]>0:
			pz_check = self.pz.copy()
			pz_check[row][col]=0
			Nof_solution = self.Solution_Count(pz_check, 0, 0)
			if Nof_solution == 1:
				self.pz[row][col]=0
				print(self.pz)
				print("{} zeros".format(sum(self.pz.flat==0)))
				print()
		self.DigHoles(randomlist, n+1)

	def createSoduku(self):
		self.SudokuSolver(0)
		print(self.pz, "--------- Answer\n")
		randomlist = list(range(81))
		random.shuffle(randomlist)
		self.DigHoles(randomlist, 0)
		print(self.pz)
		

	# def main():
	# 	puzzle = np.zeros((9,9), dtype=int)
	# 	SudokuSolver(puzzle, 0)
	# 	print(puzzle, "--------- Answer\n")
	# 	randomlist = list(range(81))
	# 	random.shuffle(randomlist)
	# 	DigHoles(puzzle, randomlist, 0)
	# 	return puzzle

NaiveS = NaiveSodukuGenerator()
NaiveS.createSoduku()

[[7 9 5 3 8 6 2 1 4]
 [6 8 1 2 4 9 7 5 3]
 [2 4 3 5 7 1 9 6 8]
 [8 7 6 9 5 4 1 3 2]
 [3 5 9 8 1 2 4 7 6]
 [1 2 4 6 3 7 5 8 9]
 [9 1 7 4 6 8 3 2 5]
 [5 6 2 7 9 3 8 4 1]
 [4 3 8 1 2 5 6 9 7]] --------- Answer

[[7 9 5 3 8 6 2 1 4]
 [6 8 1 2 4 9 7 5 3]
 [2 4 3 5 7 1 9 6 8]
 [8 7 6 9 5 4 1 3 2]
 [3 5 9 8 1 2 4 7 6]
 [1 2 4 6 3 7 5 8 9]
 [9 1 7 4 6 8 3 2 5]
 [5 6 2 7 9 3 8 4 1]
 [4 3 8 1 2 5 6 9 7]]


In [53]:
# 作者：Icybee

import copy
import random
class Sudoku:
    # @param board, a 9x9 2D array
    # Solve the Sudoku by modifying the input board in-place.
    # Do not return any value.
    def __init__(self):
        self.grids = [(i,j) for j in range(9) for i in range(9)]
        self.peercross = [[[[(j,i) for i in range(9)],[(i,k) for i in range(9)],[(j - j % 3 + i // 3,k - k % 3 + i % 3)for i in range(9)]] for k in range(9)]for j in range(9)]
        self.peers = [[set(sum(self.peercross[j][k],[])) - set([(j,k)]) for k in range(9)] for j in range(9)]
        self.finalsolution = False
    
    # convert Line to list
    def oneLine2List(self, line):
        if len(line) != 81:
            print("Unable to Load Line(Length !=81)")
            return None
        else:
            board = []
            for i in range(0,81,9):
                board.append(line[i:i+9])
            return board

    def solveSudoku(self, board):
        if not isinstance(board, list):
            board = self.oneLine2List(board)
        self.finalsolution = False
        possiblenumbers = [[[i for i in range(1,10)]for j in range(9) ]for k in range(9)]
        assert (len(board) == 9 and all((len(board[i]) == 9) for i in range(9))) == True
        if not self.preProcess(board,possiblenumbers):return False
        self.reSlove(possiblenumbers,1)
        return self.finalsolution

    def preProcess(self,board,possiblenumbers):
        for i in range(81):
            if not self.fill(i // 9,i % 9,board[i // 9][i % 9],possiblenumbers):
                return False
        return True
    #@return whether the value can be filled in the current grid
    def fill(self,x,y,value,possiblenumbers):
        if value not in range(1,10) and value not in "123456789":return True
        if int(value) not in possiblenumbers[x][y] or x not in range(9) or y not in range(9):return False
        if len(possiblenumbers[x][y]) == 1 or all([self.eliminate(x,y,i,possiblenumbers) for i in  [j for j in possiblenumbers[x][y] if j != int(value)]]):
            return True
        return False
    #@return whether elimatesuccess or not
    def eliminate(self,x,y,value,possiblenumbers):
        if value not in possiblenumbers[x][y]:
            return True
        if len(possiblenumbers[x][y]) == 1:
            return False
        possiblenumbers[x][y].remove(value)
        #stg1
        if len(possiblenumbers[x][y]) == 0 or len(possiblenumbers[x][y]) == 1 and  not all([self.eliminate(i,j,possiblenumbers[x][y][0],possiblenumbers) for (i,j) in self.peers[x][y] if (possiblenumbers[x][y][0]) in possiblenumbers[i][j]]):
             return False
        #stg2
        for k in self.peercross[x][y]:
            tem = [(i,j) for (i,j) in k if value in possiblenumbers[i][j]]
            if len(tem) == 1 and len(possiblenumbers[tem[0][0]][tem[0][1]]) > 1 and  not self.fill(tem[0][0],tem[0][1],value,possiblenumbers):
                return False
        return True

    def reSlove(self,possiblenumbers,deepth):
        if all(len(possiblenumbers[i][j]) == 1 for (i,j) in self.grids):
            if not self.finalsolution:
                self.finalsolution = copy.deepcopy(possiblenumbers)
            else:
                self.finalsolution = True
            return
        length,x,y = min((len(possiblenumbers[i][j]),i,j) for (i,j) in self.grids if len(possiblenumbers[i][j]) > 1)
        posibility = copy.copy(possiblenumbers[x][y])
        for i in posibility:
            copyindex = copy.deepcopy(possiblenumbers)
            if self.fill(x,y,i,copyindex) and self.finalsolution != True:
                self.reSlove(copyindex,deepth + 1)
        return

    def generateSolvedSudo(self):
        self.finalsolution = False
        possiblenumbers = [[[i for i in range(1,10)] for j in range(9) ]for k in range(9)]
        self.reGenerate(possiblenumbers)
        return [[str(self.finalsolution[i][j][0]) for j in range(9)] for i in range(9)]

    def reGenerate(self,possiblenumbers):
         if all(len(possiblenumbers[i][j]) == 1 for (i,j) in self.grids):
            self.finalsolution = copy.deepcopy(possiblenumbers)
            return True
         length,x,y = random.choice([(len(possiblenumbers[i][j]),i,j) for (i,j) in self.grids if len(possiblenumbers[i][j]) == (min(len(possiblenumbers[x][y]) for (x,y) in self.grids if len(possiblenumbers[x][y]) > 1))])
         posibility = copy.copy(possiblenumbers[x][y])
         for i in posibility:
            copyindex = copy.deepcopy(possiblenumbers)
            if self.fill(x,y,i,copyindex) and self.finalsolution != True:
                if self.reGenerate(copyindex):
                    return True
         return False

    #The most simple way to generate a puzzle
    def generateUnsolvedPuzzle(self):
        solvedPuzzle = self.generateSolvedSudo()
        ran = copy.deepcopy(self.grids)
        random.shuffle(ran)
        for (i,j) in ran:
            buffer = solvedPuzzle[i][j]
            solvedPuzzle[i][j] = '.'
            if self.solveSudoku(solvedPuzzle) == True:
                solvedPuzzle[i][j] = buffer
        return [''.join(i) for i in solvedPuzzle]

    #a little complicated way to generate a symetry sudoky puzzle
    def generateCentralSymmetryPuzzle(self):
        solvedPuzzle = self.generateSolvedSudo()
        ran = copy.deepcopy([self.grids[i] for i in range(len(self.grids) // 2 + 1)])
        random.shuffle(ran)
        for (i,j) in ran:
            buffer1 = solvedPuzzle[i][j]
            buffer2 = solvedPuzzle[8 - i][8 - j]
            solvedPuzzle[i][j] = '.'
            solvedPuzzle[8 - i][8 - j] =  '.'
            if self.solveSudoku(solvedPuzzle) == True:
                solvedPuzzle[i][j] = buffer1
                solvedPuzzle[8 - i][8 - j] = buffer2
        return [''.join(i) for i in solvedPuzzle]
        
    #show the puzzle
    def show(self,puzzle):
        for i in range(len(puzzle)):
            for j in range(len(puzzle[i])):
                if puzzle[i][j] is chr:
                    print(puzzle[i][j],end = '  ')
                else:
                    print(puzzle[i][j][0], end = '  ')
                if j == 2 or j == 5 or j == 8:
                    print('|' ,end = "  ")
            print("")
            if i == 2 or i == 5:
                print('-' * 34)
        print("")
s = Sudoku()
# print('--------------answer-------------')
    #print  s.solveSudoku(['.7.8....2', '.6.9...3.', '.......47', '....1..89', '.8.4.....', '.....2...', '7.93.....', '1...75.9.', '3.......1'])
    # puzzle = s.generateCentralSymmetryPuzzle()
# puzzle = s.generateUnsolvedPuzzle()
# s.show(puzzle)
# print(puzzle)
# sudoku_string_1 = "030467050920010006067300148301006027400850600090200400005624001203000504040030702"
sudoku_string_1 = "004700020200008000000260010000900102100000005709003000030054000000800003020007800"
sudoku_string_2 = "800000000003600000070090200050007000000045700000100030001000068008500010090000400"
sudoku_string_2 = sudoku_string_2.replace("0", ".")
# %prun s.show(s.solveSudoku(sudoku_string_1))
%prun s.solveSudoku(sudoku_string_2)

         872194 function calls (723383 primitive calls) in 0.803 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
88222/321    0.173    0.000    0.404    0.001 copy.py:128(deepcopy)
    34545    0.116    0.000    0.116    0.000 2743691208.py:60(<listcomp>)
29211/321    0.110    0.000    0.397    0.001 copy.py:200(_deepcopy_list)
19430/441    0.097    0.000    0.358    0.001 2743691208.py:49(eliminate)
 6963/364    0.089    0.000    0.345    0.001 2743691208.py:56(<listcomp>)
   116800    0.035    0.000    0.035    0.000 {method 'append' of 'list' objects}
    29211    0.028    0.000    0.046    0.000 copy.py:242(_keep_alive)
   176604    0.026    0.000    0.026    0.000 {method 'get' of 'dict' objects}
 3537/401    0.023    0.000    0.364    0.001 2743691208.py:42(fill)
    161/1    0.022    0.000    0.778    0.778 2743691208.py:65(reSlove)
   146965    0.021    0.000    0.021    0.000 {built-in method builtins.id}
   122188

0
9
18
27
36
45
54
63
72


In [2]:
import copy
class Solution:
    # @param board, a 9x9 2D array
    # Solve the Sudoku by modifying the input board in-place.
    # Do not return any value.
    def cross(self,A,B):
        "Cross product of elements in A amd elements in B"
        return [a + b for a in A for b in B]
    def __init__(self):
        self.grids = [(i,j) for j in range(9) for i in range(9)]
        self.peercross = [[[[(j,i) for i in range(9)],[(i,k) for i in range(9)],[(j - j % 3 + i // 3,k - k % 3 + i % 3)for i in range(9)]] for k in range(9)]for j in range(9)]
        self.peers = [[set(sum(self.peercross[j][k],[])) - set([(j,k)]) for k in range(9)] for j in range(9)]
        self.finalsolution = False
    def solveSudoku(self, board):
        self.finalsolution = False
        possiblenumbers = [[[i for i in range(1,10)]for j in range(9) ]for k in range(9)]
        assert (len(board) == 9 and all((len(board[i]) == 9) for i in range(9))) == True
        if not self.preProcess(board,possiblenumbers):return False
        self.reSlove(possiblenumbers,1)
        return self.finalsolution
    def preProcess(self,board,possiblenumbers):
        for i in range(81):
            if not self.fill(i // 9,i % 9,board[i // 9][i % 9],possiblenumbers):
                return False
        return True
    #@return whether the value can be filled in the current grid
    def fill(self,x,y,value,possiblenumbers):
        if value not in range(1,10) and value not in "123456789":return True
        if int(value) not in possiblenumbers[x][y] or x not in range(9) or y not in range(9):return False
        if len(possiblenumbers[x][y]) == 1 or all([self.eliminate(x,y,i,possiblenumbers) for i in  [j for j in possiblenumbers[x][y] if j != int(value)]]):
            return True
        return False
    #@return whether elimatesuccess or not
    def eliminate(self,x,y,value,possiblenumbers):
        if value not in possiblenumbers[x][y]:
            return True
        if len(possiblenumbers[x][y]) == 1:
            return False
        possiblenumbers[x][y].remove(value)
        #stg1
        if len(possiblenumbers[x][y]) == 0 or len(possiblenumbers[x][y]) == 1 and  not all([self.eliminate(i,j,possiblenumbers[x][y][0],possiblenumbers) for (i,j) in self.peers[x][y] if (possiblenumbers[x][y][0]) in possiblenumbers[i][j]]):
             return False
        #stg2
        for k in self.peercross[x][y]:
            tem = [(i,j) for (i,j) in k if value in possiblenumbers[i][j]]
            if len(tem) == 1 and len(possiblenumbers[tem[0][0]][tem[0][1]]) > 1 and  not self.fill(tem[0][0],tem[0][1],value,possiblenumbers):
                return False
        return True
    def reSlove(self,possiblenumbers,deepth):
        if all(len(possiblenumbers[i][j]) == 1 for (i,j) in self.grids):
            if not self.finalsolution:
                self.finalsolution = copy.deepcopy(possiblenumbers)
            else:
                self.finalsolution = True
            return
        length,x,y = min((len(possiblenumbers[i][j]),i,j) for (i,j) in self.grids if len(possiblenumbers[i][j]) > 1)
        posibility = copy.copy(possiblenumbers[x][y])
        for i in posibility:
            copyindex = copy.deepcopy(possiblenumbers)
            if self.fill(x,y,i,copyindex) and self.finalsolution != True:
                self.reSlove(copyindex,deepth + 1)
        #print('deepth: %s ' %deepth)
        return
s = Solution()
print('-------answer--------')
print(s.solveSudoku(["..9748...","7........",".2.1.9...","..7...24.",".64.1.59.",".98...3..","...8.3.2.","........6","...2759.."]))


-------answer--------
[[[5], [1], [9], [7], [4], [8], [6], [3], [2]], [[7], [8], [3], [6], [5], [2], [4], [1], [9]], [[4], [2], [6], [1], [3], [9], [8], [7], [5]], [[3], [5], [7], [9], [8], [6], [2], [4], [1]], [[2], [6], [4], [3], [1], [7], [5], [9], [8]], [[1], [9], [8], [5], [2], [4], [3], [6], [7]], [[9], [7], [5], [8], [6], [3], [1], [2], [4]], [[8], [3], [2], [4], [9], [1], [7], [5], [6]], [[6], [4], [1], [2], [7], [5], [9], [8], [3]]]


In [39]:
from time import perf_counter
from copy import copy

# SUDOKU GRID
sudoku = [[0, 7, 0, 9, 4, 2, 0, 1, 0],
          [0, 0, 8, 5, 0, 1, 0, 0, 0],
          [0, 9, 0, 0, 0, 0, 0, 0, 0],
          [8, 3, 0, 0, 0, 0, 0, 0, 0],
          [6, 5, 7, 2, 0, 0, 0, 8, 0],
          [0, 2, 9, 0, 0, 0, 0, 7, 0],
          [0, 8, 0, 1, 9, 0, 0, 0, 0],
          [9, 0, 0, 0, 0, 7, 1, 2, 5],
          [5, 0, 0, 6, 2, 3, 8, 0, 7]]

# HEADER GENERATION

def generateColumns(size):
    size += 1
    row_column = []
    for i in range(1, size):
        for j in range(1, size):
            row_column.append('R'+str(i)+'C'+str(j))
    row_number = []
    for i in range(1,size):
        for j in range(1, size):
            row_number.append('R'+str(i)+'#'+str(j))
    column_number = []
    for i in range(1, size):
        for j in range(1, size):
            column_number.append('C'+str(i)+'#'+str(j))
    box_number = []
    for i in range(1, size):
        for j in range(1, size):
            box_number.append('B'+str(i)+'#'+str(j))

    return row_column + row_number + column_number + box_number

# ROW GENERATION
# DOES NOT CURRENTLY WORK FOR NUMBERS OTHER THAN 9
def generateRows(size, X):
    final = []
    for row in range(size):
        for column in range(size):
            for cell in range(size):
                temp = []
                # Add Row-Column
                temp.append(X[row*size + column])
                # Add Row-Number
                temp.append(X[size**2 + row*size + cell])
                # Add Column-Number
                temp.append(X[((size**2)*2) + column*size + cell])
                # Add Box-Number
                box = 0
                if row > 2:
                    box += 3
                if row > 5:
                    box += 3
                if column > 2:
                    box += 1
                if column > 5:
                    box += 1
                temp.append(X[((size**2)*3) + box*size + cell])
                final.append(temp)
    return final

class Node:

    def __init__(self):
        self.right = None
        self.left = None
        self.up = None
        self.down = None
        self.column = None

class ColumnNode:

    def __init__(self, name):
            self.name = name
            self.right = None
            self.left = None
            self.up = None
            self.down = None
            self.size = None

class SudokuSolver:

    def __init__(self, sudoku):
        self.header = ColumnNode('Header')
        self.sudoku = sudoku
        self.constraints = generateColumns(9)
        self.rows = generateRows(9, self.constraints)

        self.solution = {}
        self.solution_key = 0

        
        # For each constraint, make a column.
        tic = perf_counter()
        temp_column = self.header
        for column in self.constraints:
            temp_column.right = ColumnNode(column)
            temp_column.right.left = temp_column
            temp_column = temp_column.right
        temp_column.right = self.header
        self.header.left = temp_column

        # For each row, go across and add one node down.
        for row in self.rows:
            first_node = None
            previous_node = None
            for item in row:
                temp_column = self.header.right
                while temp_column != self.header:
                    if item == temp_column.name:
                        #print("Placing a node in Node " + temp_column.name)
                        temp = temp_column
                        while temp.down != None:
                            temp = temp.down
                        temp.down = Node()
                        if first_node == None:
                            first_node = temp.down
                        if previous_node != None:
                            temp.down.left = previous_node
                            temp.down.left.right = temp.down
                        temp.down.up = temp
                        temp.down.column = temp_column
                        previous_node = temp.down
                        break
                    temp_column = temp_column.right
            first_node.left = previous_node
            previous_node.right = first_node

            #print('\n')

        # Link up each bottom node with their header node.
        temp = self.header.right
        while temp != self.header:
            size = 0
            while temp.down != None:
                size += 1
                temp = temp.down
            temp.column.size = size
            temp.down = temp.column
            temp.down.up = temp
            temp = temp.down.right

        toc = perf_counter()
        self.SolvedLinkRemoval()
        print(f"Linked list made in {toc - tic:0.4f} seconds.")

    def SolvedLinkRemoval(self):

        self.solved_parts = []
        self.rows_to_remove = []

        for row in self.sudoku:
            for digit in row:
                if digit != 0:
                    self.solved_parts.append((self.sudoku.index(row) + 1, row.index(digit) + 1, digit))

        for row, column, digit in self.solved_parts:
            temp = self.header.right
            temp_name = 'R'+str(row)+'C'+str(column)
            while temp != self.header:
                if temp.name == temp_name:
                    for i in range(digit):
                        temp = temp.down
                    self.rows_to_remove.append(temp)
                    break
                temp = temp.right

        for node in self.rows_to_remove:
            self.CoverColumn(node.column)
            j = node.right
            while j != node:
                self.CoverColumn(j.column)
                j = j.right

    def CoverColumn(self, c):
        c.right.left = c.left
        c.left.right = c.right
        i = c.down
        while i != c:
            j = i.right
            while j != i:
                j.down.up = j.up
                j.up.down = j.down
                j.column.size = j.column.size - 1
                j = j.right
            i = i.down


    def UncoverColumn(self, c):
        i = c.up
        while i != c:
            j = i.left
            while j != i:
                j.column.size = j.column.size + 1
                j.down.up = j
                j.up.down = j
                j = j.left
            i = i.up
        c.right.left = c
        c.left.right = c

    def SudokuSolve(self, k=0, solution={}, size_heuristic=False):

        if self.header == self.header.right:
            self.solution[self.solution_key] = []
            for o in solution.values():
                self.solution[self.solution_key].append(o.column.name + o.right.column.name[-2:])
            self.solution_key += 1
            return
        else:
            # Choose column c...
            if size_heuristic: 
                s = float('inf')
                j = self.header.right
                while j != self.header:
                    if j.size < s:
                        c = j
                        s = j.size
                    j = j.right
            else:
                c = self.header.right
            # ...and cover it.
            self.CoverColumn(c)
            # For each r below c, make r the k-th part of the solution.
            r = c.down
            while r != c:
                solution[k] = r

                j = r.right
                while j != r:
                    self.CoverColumn(j.column)
                    j = j.right

                self.SudokuSolve(k=k+1, solution=solution)

                r = solution[k]
                c = r.column

                j = r.left
                while j != r:
                    self.UncoverColumn(j.column)
                    j = j.left

                r = r.down
            
            self.UncoverColumn(c)

            if k == 0:
                return (self.solution)

            return

    def FillInSudoku(self, solutions_dict):

        number = 1
        for solution in solutions_dict.values():
            for cell in solution:
                # cell is a string like 'R1C1#1'
                # cell[1] is the row + 1
                row = int(cell[1]) - 1
                # cell[3] is the column + 1
                column = int(cell[3]) - 1
                # cell[5] is the digit
                digit = int(cell[5])
                self.sudoku[row][column] = digit
            print(f'Solution #{number}')
            for i in self.sudoku:
                print(i)
            print('')
            number += 1


# Test = "800000000003600000070090200050007000000045700000100030001000068008500010090000400"
sudoku = [
    [8,0,0,0,0,0,0,0,0],
    [0,0,3,6,0,0,0,0,0],
    [0,7,0,0,9,0,2,0,0],
    [0,5,0,0,0,7,0,0,0],
    [0,0,0,0,4,5,7,0,0],
    [0,0,0,1,0,0,0,3,0],
    [0,0,1,0,0,0,0,6,8],
    [0,0,8,5,0,0,0,1,0],
    [0,9,0,0,0,0,4,0,0]
]
llist = SudokuSolver(sudoku)
%prun llist.FillInSudoku(llist.SudokuSolve())

Linked list made in 0.3249 seconds.


KeyboardInterrupt: 

In [50]:
import random 
import sys  
sys.setrecursionlimit(100000) # 发现python默认的递归深度是很有限的
                              #（默认是1000），因此当递归深度超过999的
                              # 样子，就会引发这样的一个异常。


def get_next(m:"数独矩阵", x:"空白格行数", y:"空白格列数"):
    """ 功能：获得下一个空白格在数独中的坐标。       
    """
    for next_y in range(y+1, 9):  # 下一个空白格和当前格在一行的情况
        if m[x][next_y] == 0:
            return x, next_y
    for next_x in range(x+1, 9):  # 下一个空白格和当前格不在一行的情况
        for next_y in range(0, 9):
            if m[next_x][next_y] == 0:
                return next_x, next_y
    return -1, -1               # 若不存在下一个空白格，则返回 -1，-1

def value(m:"数独矩阵", x:"空白格行数", y:"空白格列数"):
    """ 功能：返回符合"每个横排和竖排以及
              九宫格内无相同数字"这个条件的有效值。
    """ 
    i, j = x//3, y//3
    grid = [m[i*3+r][j*3+c] for r in range(3) for c in range(3)]
    v = set([x for x in range(1,10)]) - set(grid) - set(m[x]) - \
        set(list(zip(*m))[y])    
    return list(v)

def start_pos(m:"数独矩阵"):
    """ 功能：返回第一个空白格的位置坐标"""
    for x in range(9):
        for y in range(9):
            if m[x][y] == 0:
                return x, y
    return False, False  # 若数独已完成，则返回 False, False

def try_sudoku(m:"数独矩阵", x:"空白格行数", y:"空白格列数"):
    """ 功能：试着填写数独 """
    for v in value(m, x, y):
        m[x][y] = v
        next_x, next_y = get_next(m, x, y)
        if next_y == -1: # 如果无下一个空白格
            return True
        else:
            end = try_sudoku(m, next_x, next_y) # 递归
            if end:
                return True
            m[x][y] = 0 # 在递归的过程中，如果数独没有解开，
                        # 则回溯到上一个空白格

def sudoku(m):        
    x, y = start_pos(m)
    try_sudoku(m, x, y)
    print(m)     
    
if __name__ == "__main__":
    sudoku2 = [
        [8,0,0,0,0,0,0,0,0],
        [0,0,3,6,0,0,0,0,0],
        [0,7,0,0,9,0,2,0,0],
        [0,5,0,0,0,7,0,0,0],
        [0,0,0,0,4,5,7,0,0],
        [0,0,0,1,0,0,0,3,0],
        [0,0,1,0,0,0,0,6,8],
        [0,0,8,5,0,0,0,1,0],
        [0,9,0,0,0,0,4,0,0]
    ]

    %prun sudoku(sudoku2)



[[8, 1, 2, 7, 5, 3, 6, 4, 9], [9, 4, 3, 6, 8, 2, 1, 7, 5], [6, 7, 5, 4, 9, 1, 2, 8, 3], [1, 5, 4, 2, 3, 7, 8, 9, 6], [3, 6, 9, 8, 4, 5, 7, 2, 1], [2, 8, 7, 1, 6, 9, 5, 3, 4], [5, 2, 1, 9, 7, 4, 3, 6, 8], [4, 3, 8, 5, 2, 6, 9, 1, 7], [7, 9, 6, 3, 1, 8, 4, 5, 2]]
 

         309848 function calls (247885 primitive calls) in 1.019 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    61964    0.542    0.000    0.804    0.000 354830788.py:20(value)
    61964    0.225    0.000    0.225    0.000 354830788.py:25(<listcomp>)
  61964/1    0.152    0.000    1.017    1.017 354830788.py:38(try_sudoku)
    61964    0.061    0.000    0.061    0.000 354830788.py:8(get_next)
    61964    0.037    0.000    0.037    0.000 354830788.py:26(<listcomp>)
        1    0.001    0.001    0.001    0.001 socket.py:543(send)
        1    0.000    0.000    1.019    1.019 {built-in method builtins.exec}
        2    0.000    0.000    0.001    0.000 iostream.py:525(write)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.print}
        1    0.000    0.000    0.001    0.001 iostream.py:209(schedule)
        1    0.000    0.000    1.018    1.018 354830788.py:52(sudoku)
        2    0.000    0.000 

In [2]:
from math import inf

class Node(object):
    def __init__(self, column, row):
        self.column = column
        self.row = row
        self.up, self.down, self.left, self.right = self, self, self, self

class ColumnNode(Node):
    def __init__(self, id):
        Node.__init__(self, self, id)
        self.row_count = 0

class DLX(object):
    def __init__(self):
        self.root = ColumnNode(0)

    def create_matrix(self, grid_str):
        """Creates an exact cover matrix from a sudoku grid"""
        root = self.root
        cols = [root]
        # Construct column headers as a doubly circular linked list
        # We'll be storing all the column headers in a list for easy access
        for i in range(324):
            c = ColumnNode(i + 1)
            c.right = root
            c.left = root.left
            root.left.right = c
            root.left = c
            cols.append(c)

        # These help us find which constraint should be filled in
        row_constraint = lambda x, k: 81 + (x // 9) * 9 + k
        col_constraint = lambda x, k: 162 + (x % 9) * 9 + k
        box_constraint = lambda x, k: 243 + (x // 27) * 27 + (x % 9) // 3 * 9 + k
        row_num = lambda x, k: x * 9 + k

        def _append_to_column(n):
            """Appends a row node at the end of a column"""
            c = n.column
            c.row_count += 1
            n.down = c
            n.up = c.up
            c.up.down = n
            c.up = n

        def _create_links(x, k):
            """Creates links for a row"""
            cell_node = Node(cols[x + 1], row_num(x, k))
            row_node = Node(cols[row_constraint(x, k)], row_num(x, k))
            col_node = Node(cols[col_constraint(x, k)], row_num(x, k))
            box_node = Node(cols[box_constraint(x, k)], row_num(x, k))

            # Link all the nodes into a single row
            cell_node.right, cell_node.left = row_node, box_node
            row_node.right, row_node.left = col_node, cell_node
            col_node.right, col_node.left = box_node, row_node
            box_node.right, box_node.left = cell_node, col_node

            _append_to_column(cell_node)
            _append_to_column(row_node)
            _append_to_column(col_node)
            _append_to_column(box_node)


        for index, chr in enumerate(grid_str):
            if chr == '.':
                # Square is empty, add all possible values
                for k in range(9):
                    _create_links(index, k + 1)
            else:
                _create_links(index, ord(chr) - 48)

    def choose_least_column(self):
        """
        We use the S heuristic to minimize branching factor
        Returns the column with the least number of rows
        """
        c = None
        i = self.root.right
        s = inf
        while i != self.root:
            if i.row_count < s:
                c = i
                s = i.row_count
            i = i.right
        return c

    def cover(self, col):
        """Removes a column along with all rows that intersect said column"""
        col.right.left = col.left
        col.left.right = col.right
        i = col.down
        while i != col:
            # Iterate through nodes in row and unlink them
            j = i.right
            while j != i:
                j.down.up = j.up
                j.up.down = j.down
                j.column.row_count -= 1
                j = j.right
            i = i.down

    def uncover(self, col):
        """Undo covering of a column"""
        i = col.up
        while i != col:
            j = i.left
            while j != i:
                j.down.up = j
                j.up.down = j
                j.column.row_count += 1
                j = j.left
            i = i.up
        col.right.left = col
        col.left.right = col

    def search(self, solution):
        """Search for a solution from a exact cover matrix"""

        # No columns left, a solution is found
        if self.root == self.root.right:
            return solution, True

        c = self.choose_least_column()
        self.cover(c)

        i = c.down
        while i != c:
            solution.append(i)
            j = i.right
            while j != i:
                self.cover(j.column)
                j = j.right

            solution, found = self.search(solution)
            if found:
                return solution, True

            i = solution.pop()
            c = i.column
            j = i.left
            while j != i:
                self.uncover(j.column)
                j = j.left

            i = i.down

        self.uncover(c)
        return solution, False


class Sudoku(object):
    def solve(self, grid_str):
        solver = DLX()
        solver.create_matrix(grid_str)
        dlx_solution, found = solver.search([])
        return dlx_solution, found

    def output_solution(self, dlx_solution, found):
        """Converts a solution set from Dancing Links into a grid"""
        if not found:
            print('Solution not found')
            return
        solution = [0] * 81
        for i in dlx_solution:
            val = i.row % 9
            if val == 0:
                val = 9
            solution[(i.row - 1) // 9] = val
        self.output_grid(''.join(str(i) for i in solution))

    def output_grid(self, grid_str):
        """Outputs Sudoku grid in a readable format"""
        print('')
        grid = list(grid_str)
        row = list('+-------+-------+-------+')
        for index, chr in enumerate(grid):
            if index % 9 == 0:
                print(''.join(row))
                if index % 27 == 0 and index > 0:
                    print('+-------+-------+-------+')
                row = []
            if index % 3 == 0:
                row.extend(['|', ' '])
            row.extend([chr, ' '])
            if (index + 1) % 9 == 0:
                row.append('|')
            if index == len(grid_str) - 1:
                print(''.join(row))
                print('+-------+-------+-------+\n')


s = Sudoku()
# print('Enter a sudoku puzzle in the format shown below:')
# print('7......5..5.98472383..2...9.79.58.4...........6.14.97.5...3..94126495.8..4......1\n')
# while 1:
print("HH")
grid_str = "800000000003600000070090200050007000000045700000100030001000068008500010090000400"
grid_str = grid_str.replace("0", ".")
%prun solution, found = s.solve(grid_str)
# s.output_solution(solution, found)


HH
 

         27477 function calls (25985 primitive calls) in 0.181 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1492    0.098    0.000    0.098    0.000 3453978379.py:74(choose_least_column)
     5968    0.036    0.000    0.036    0.000 3453978379.py:89(cover)
     5644    0.025    0.000    0.025    0.000 3453978379.py:104(uncover)
   1493/1    0.013    0.000    0.172    0.172 3453978379.py:118(search)
      561    0.003    0.000    0.007    0.000 3453978379.py:47(_create_links)
     2569    0.002    0.000    0.002    0.000 3453978379.py:4(__init__)
     2244    0.001    0.000    0.001    0.000 3453978379.py:38(_append_to_column)
        1    0.001    0.001    0.008    0.008 3453978379.py:18(create_matrix)
     1816    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
     2244    0.000    0.000    0.000    0.000 3453978379.py:36(<lambda>)
     1411    0.000    0.000    0.000    0.000 {method 'pop' of

In [3]:
%prun 
solution, found = s.solve(grid_str)
s.output_solution(solution, found)

 
+-------+-------+-------+
| 8 1 2 | 7 5 3 | 6 4 9 |
| 9 4 3 | 6 8 2 | 1 7 5 |
| 6 7 5 | 4 9 1 | 2 8 3 |
+-------+-------+-------+
| 1 5 4 | 2 3 7 | 8 9 6 |
| 3 6 9 | 8 4 5 | 7 2 1 |
| 2 8 7 | 1 6 9 | 5 3 4 |
+-------+-------+-------+
| 5 2 1 | 9 7 4 | 3 6 8 |
| 4 3 8 | 5 2 6 | 9 1 7 |
| 7 9 6 | 3 1 8 | 4 5 2 |
+-------+-------+-------+



         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}